In [2]:
# Для работы с html файлами
from bs4 import BeautifulSoup

# import requests as req

# Для работы с таблицами
import pandas as pd

# Для работы со смайликами в тексте
import emoji

# Для превращения смайликов в слова
import demoji

# pip install googletrans==3.1.0a0
# Переводчик
from googletrans import Translator

# Для VK
from vk_api import VkApi

import json

ModuleNotFoundError: No module named 'emoji'

https://dev-gang.ru/article/perevod-teksta-s-pomosczu-google-translate-api-v-python-ahgm88wx1k/

# Парсинг вконтакте


In [64]:
import vk_api

In [3]:
# Вставьте сюда свой access_token, полученный из адресной строки!
# vk1.a.cPo9dt5bWCZbbkHqA8qiOmaly13S0rDKB91SKF8IyZIsSKlaNSjB8SEOCL4SSeeaJXkgy-Q6UrbSyxOZGZl8Mbx1Qf95X2Ppbv1RBgg5HhxDk-V4SVJD4L9NtbQQpqQ8rpy_mGw6ctdlLoBtd9klDnRjzj5sSP4oXw8k0AyALEwNNHw6BnDrbM3V_57_EGS3wWqyoTCD6ylTPkptJP_fMg


token='vk1.a.cPo9dt5bWCZbbkHqA8qiOmaly13S0rDKB91SKF8IyZIsSKlaNSjB8SEOCL4SSeeaJXkgy-Q6UrbSyxOZGZl8Mbx1Qf95X2Ppbv1RBgg5HhxDk-V4SVJD4L9NtbQQpqQ8rpy_mGw6ctdlLoBtd9klDnRjzj5sSP4oXw8k0AyALEwNNHw6BnDrbM3V_57_EGS3wWqyoTCD6ylTPkptJP_fMg'

In [ ]:
# # Функция для парсинга постов со стены выбранной группы ВК через group_id
# def main(offset: int, token: str, group_id: str):
#     vk = vk_api.VkApi(token=token) # авторизация через токен 
#     api = vk.get_api()
    
    
#     # Числовой Айди гробастаем
#     group_info = api.groups.getById(group_id=group_id)
#     numeric_group_id = -group_info[0]['id']
    
#     posts = api.wall.get(owner_id = numeric_group_id, offset = offset, count=100)['items']
#     posts_strings = [post['text'] for post in posts]
#     num_like = []
#     comments_strings = []
#     for post in posts:
#         comments = api.wall.getComments(owner_id = numeric_group_id, post_id=post['id'], count=100)['items']
#         comments_strings.append([comment['text'] for comment in comments])
#         itemID = post['id']
#         isLiked = api.likes.getList(
#         type = 'post', 
#         owner_id = numeric_group_id, 
#         item_id = itemID               
#         )
#         num_like.append(isLiked['count'])
#     return posts_strings, comments_strings, num_like

In [65]:
def main(offset: int, token: str, group_id: str, count: int):
    vk = vk_api.VkApi(token=token)
    api = vk.get_api()
    
    # Получаем числовой ID группы
    group_info = api.groups.getById(group_id=group_id)
    numeric_group_id = -group_info[0]['id']
    
    # Получаем посты
    posts = api.wall.get(owner_id=numeric_group_id, offset=offset, count=count)['items']
    
    posts_strings = []
    # num_like = []
    comments_data = []
    
    for post in posts:
        posts_strings.append(post['text'])
        
        try:
            # Получаем комментарии с информацией об авторах
            comments_response = api.wall.getComments(
                owner_id=numeric_group_id, 
                post_id=post['id'], 
                count=100,
                extended=1  # Добавляем информацию о пользователях
            )
            
            comments_items = comments_response['items']
            
            # Собираем данные комментариев с метаданными
            post_comments_data = []
            for comment in comments_items:
                comment_data = {
                    "text": comment.get('text', ''),
                    "metadata": {
                        "group_id": group_id,
                        "author_id": comment.get('from_id', '')
                    }
                }
                post_comments_data.append(comment_data)
            
            comments_data.append(post_comments_data)
            
        except Exception as e:
            print(f"Ошибка при получении комментариев: {e}")
            comments_data.append([])
        
        # # Лайки
        # try:
        #     isLiked = api.likes.getList(
        #         type='post', 
        #         owner_id=numeric_group_id,
        #         item_id=post['id']               
        #     )
        #     num_like.append(isLiked['count'])
        # except Exception as e:
        #     print(f"Ошибка при получении лайков: {e}")
        #     num_like.append(0)
    
    return posts_strings, comments_data

In [ ]:
# combo_list_posts = []
# combolist_comments = []
# combolist_like_count = []
# for i in range(100, 301, 100):
#     try:
#         rzd_posts, comments_strings_rzd, rzd_like_count = main(offset = i, token = token, group_id = 'pmmgo')
#         combo_list_posts.extend(rzd_posts)
#         combolist_comments.extend(comments_strings_rzd)
#         combolist_like_count.extend(rzd_like_count)
#     except:
#         print('Постов больше нет на смещении: ', i)

In [ ]:
# Проверил, поста рял из группы
# combo_list_posts[:5]

['Ⓜ Проект станции метро «ЗИЛ» на Бирюлёвской линии. На ветке будет 10 станций. \n\nВ 2028 году хотят завершить первый участок от «ЗИЛ» до «Курьяново».',
 '🤔Что думаете?',
 '🤦\u200d♂Зацеперы зацепились за передний вагон иволги',
 '🚇 40 новеньких вагонов выйдут на зеленую ветку Московского метро\n\nПосле испытаний и обкатки пассажиров поступят самые современные «Москва-2026». Сохранили преимущества предыдущих моделей, но доработали внешний вид фар и сиденья стали удобнее.',
 '😁Зацеперы начали терять хватку — прекрасные новости из Алтуфьевского.\n\nМалолетки в балаклавах пытались присосаться к 282-му автобусу, пока тот снижал скорость перед лежачим полицейским. \n\nНе получилось, не фартануло.']

In [ ]:
# Тоже проверил что все норм
# combolist_comments[:1]

[['Крутой там квартал отстроили на ЗИЛе',
  'Смотрится очень и очень красиво!!!! \nТеперь главное, чтобы было удобно переходить на МЦК и станцию ЗиЛ....']]

In [ ]:
# len(set(combo_list_posts))

292

In [ ]:
# # Добавьте функцию для сохранения данных в csv, иначе потеряете их
# my_text_1 = pd.DataFrame(data = combo_list_posts)
# # my_text_1['goal'] = 1
# my_text_1.tail()

,0,goal
295,В китайском Синине пассажиры перед поездкой са...,1
296,😅 Ворона учится основам футбола у станции метр...,1
297,"Поручик Ржевский в редкие минуты отдыха, когда...",1
298,,1
299,«Черная мамба» только вчера вышла на линию в п...,1


In [ ]:
# Один из способов превращения двумерного списка в одномерный
# comments = [message for post in combolist_comments for message in post]

In [ ]:
# len(comments)

7501

 ## Тут код Игната начинается <3
 ## Анализ тоннальности комментов метр

### Сбор данных

У нас из 300 постов получилось 7500 комментов, что нехило

Думаю для разнообразия лучше взять хотяб из 3 групп комменты, из 100 постов в каждой группе

Группы, которые взял:

* Подслушано Метро Москвы
    * id='pmmgo'

* Московское метро
    * id='mosmetro'

* Мир метро
    * id='mirmetro'

In [ ]:
# combo_list_comments = []

# for group in ['pmmgo', 'mosmetro', 'mirmetro']:
#     try:
#         rzd_posts, comments_strings_rzd, rzd_like_count = main(offset = 200, token = token, group_id = group)
#         combo_list_comments.extend(comments_strings_rzd)
#     except:
#         print('Постов больше нет на смещении: ', i)

# comments = [message for post in combo_list_comments for message in post]
# comments[:10]
# print(len(comments))

In [66]:
all_comments_data = []

for group_name in ['pmmgo', 'mosmetro', 'mirmetro']:
    
    for i in range(100, 401, 100):
        print(f"Обработка группы: {group_name} На сотке: {i}")
    
        # Оффсет сделал 1, на случай всяких рекламных постов в начале
        # В pmmgo комменты оставляют намного чаще, шобы дисбаланса небыло берем там меньше постов
        if group_name == 'pmmgo' and i==400:
            continue
        else:
            rzd_posts, comments_data_rzd = main(offset=i, token=token, group_id=group_name, count=100)

        for post_comments in comments_data_rzd:
            all_comments_data.extend(post_comments)
        

print(f"\nВсего комментов: {len(all_comments_data)}")

Обработка группы: pmmgo На сотке: 100
Обработка группы: pmmgo На сотке: 200
Обработка группы: pmmgo На сотке: 300
Обработка группы: pmmgo На сотке: 400
Обработка группы: mosmetro На сотке: 100
Обработка группы: mosmetro На сотке: 200
Обработка группы: mosmetro На сотке: 300
Обработка группы: mosmetro На сотке: 400
Обработка группы: mirmetro На сотке: 100
Обработка группы: mirmetro На сотке: 200
Обработка группы: mirmetro На сотке: 300
Обработка группы: mirmetro На сотке: 400

Всего комментов: 14262


In [67]:
# Сохраняем в файл
with open('data/vk_comments.json', 'w', encoding='utf-8') as f:
    json.dump(all_comments_data, f, ensure_ascii=False, indent=2)

print("Данные сохранены")

Данные сохранены


In [68]:
with open('data/vk_comments.json', 'r', encoding='utf-8') as f:
    loaded_data = json.load(f)

for i, comment in enumerate(loaded_data[:3], 1):
    print(f"\nКомментарий {i}:")
    print(f"  Текст: {comment['text'][:100]}...")
    print(f"  Группа: {comment['metadata']['group_id']}")
    print(f"  Автор: {comment['metadata']['author_id']}")


Комментарий 1:
  Текст: ...
  Группа: pmmgo
  Автор: 43734673

Комментарий 2:
  Текст: Где то Джерри хулюганит😁...
  Группа: pmmgo
  Автор: 863821330

Комментарий 3:
  Текст: Милый рыжик!...
  Группа: pmmgo
  Автор: 27551408


У нас получилось почти 7к комментов

В метадату я думаю добавлю айди группы, и то, активный автор или нет (если больше 10 комментов = 1, если меньше = 0)

In [69]:
from collections import Counter

In [70]:
author_counts = Counter()
for comment in loaded_data:
    author_id = comment['metadata']['author_id']
    author_counts[author_id] += 1

In [71]:
author_counts

Counter({840964348: 178,
         349012778: 128,
         695761943: 112,
         390450495: 91,
         88470611: 72,
         0: 65,
         11062102: 61,
         2701352: 57,
         27551408: 53,
         73009045: 53,
         147227048: 49,
         720604978: 47,
         852529207: 42,
         3664400: 42,
         2173273: 40,
         510403005: 39,
         251128063: 36,
         597562095: 33,
         35884108: 32,
         567177772: 31,
         585098696: 29,
         2281437: 28,
         634772283: 28,
         7696068: 27,
         707288346: 26,
         893644094: 25,
         225979593: 25,
         896915470: 25,
         829288931: 23,
         658307126: 23,
         204935776: 22,
         863821330: 21,
         1057997148: 21,
         447607869: 21,
         268307812: 21,
         31356699: 21,
         736255968: 21,
         391886190: 20,
         236105994: 20,
         639716916: 20,
         248834296: 19,
         69753777: 19,
         4303

In [72]:
# Считаем авторов с >10 комментариями
authors_more_than_10 = sum(1 for count in author_counts.values() if count >= 10)
authors_more_than_5 = sum(1 for count in author_counts.values() if count >= 5)
total_authors = len(author_counts)

# Процент
print(f"Процент авторов у которых >= 10: {authors_more_than_10 / total_authors}")
print(f"Процент авторов у которых >= 5: {authors_more_than_5 / total_authors}")

Процент авторов у которых >= 10: 0.02904830685283261
Процент авторов у которых >= 5: 0.09083614187128872


У авторов, у которых >= 10 комментов их всего полтора процента, а у которых 5 комментов 6%

Меньше 5 комментов думаю не стоит брать, т.к. числа уже совсем маленькие получатся и 4 коммента автор мог под одним обсуждаемым постом просто оставить, что значит, что вряд ли он активный прям комментатор, а просто под одним постом бомбанул, поэтому остваим порог 5

In [73]:
for comment in loaded_data:
    author_id = comment['metadata']['author_id']
    
    # Если автор оставил >= 5 комментариев, ставим 1, иначе 0
    is_frequent = 1 if author_counts[author_id] >= 5 else 0
    comment['metadata']['is_frequent_poster'] = is_frequent

In [74]:
with open('data/vk_comments_with_metadata.json', 'w', encoding='utf-8') as f:
    json.dump(loaded_data, f, ensure_ascii=False, indent=2)

## Предобработка текста

Я буду использовать модель на BERT, поэтому лемматизацию и стоп-слова с пунктуацией не убираем

Обработать эмодзи и всякий мусор на подобии HTML-тегов только надо

In [75]:
import json
import re
import html
from typing import List, Dict
import unicodedata

In [76]:
def clean_html(text: str) -> str:
    
    # HTML
    if not text:
        return ""
    
    # Декодируем HTML (&amp; → &)
    text = html.unescape(text)
    
    # Удаляем HTML-теги
    text = re.sub(r'<[^>]+>', ' ', text)
    return text

def remove_urls_and_emails(text: str) -> str:

    # URL
    text = re.sub(
        r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+',
        ' [URL] ',
        text
    )
    # www-адреса
    text = re.sub(r'www\.[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}', ' [URL] ', text)
    # Email
    text = re.sub(r'[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}', ' [EMAIL] ', text)
    return text

def normalize_whitespace(text: str) -> str:
    """Убирает лишние пробелы и переносы строк"""
    # Заменяем переносы и табы на пробелы
    text = text.replace('\n', ' ').replace('\r', ' ').replace('\t', ' ')
    # Убираем множественные пробелы
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

def process_emojis_and_emoticons(text: str) -> str:
    """Заменяет эмодзи и эмотиконы на текстовые метки"""
    # Простые замены эмодзи (можно расширить)
    emoji_replacements = {
    '😅': ' [смех] ', '😂': ' [смех] ', '😊': ' [улыбка] ',
    '😢': ' [грусть] ', '😡': ' [злость] ', '❤️': ' [сердце] ',
    '👍': ' [палец_вверх] ', '👎': ' [палец_вниз] ', '🚇': ' [метро] ',
    '🚆': ' [поезд] ', '🕒': ' [время] ', '😁': ' [радость] ',
    '😐': ' [нейтрально] ', '🤔': ' [задумчивость] ', '🙏': ' [спасибо] ',
    '😆': ' [смех] ', '💚': ' [сердце] ', '🕊️': ' [птичка] ',
    '🐥': ' [цыпленок] ', '🐣': ' [цыпленок] ', '🔥': ' [огонь] ',
    '🤣': ' [очень_смешно] ', '😀': ' [радостная_улыбка] ', '🧐': ' [задумчивость]',
    '🤨': ' [непонимание] ', '😏': ' [хитрость] ', '🙂': ' [улыбка] ',
    '🥲': ' [сдерживание] ', '🤦‍♂': ' [стыд] ', '🤷‍♂️': ' [недопонимание] ',
    '😉': ' [подмигивание] ', '🥺': ' [глазки] ', '😓': ' [тяжело] ',
    '🤗': ' [обнимание] ', '☺': ' [улыбка] ', '✊': ' [кулак] ',
    '🎉': ' [праздник] ', '🤢': ' [тошнота] ', '💕': ' [влюбленность] ',
    '😒': ' [недовольство] ', '🥹': ' [слезы_радости] ', '🤑': ' [жадность] ',
    '😤': ' [надменность] ', '😵‍💫': ' [головокружение] ', '🤧': ' [чихание] ',
    '🤯': ' [взрыв_мозга] ', '😭': ' [плач] ', '😍': ' [обожание] ',
    '☺️': ' [мягкая_улыбка] ', '😘': ' [воздушный_поцелуй] ', '😩': ' [истощение] ',
    '😔': ' [печаль] ', '😳': ' [смущение] ', '🤬': ' [сильная_злость] ',
    '🥶': ' [холод] ', '😱': ' [крик_страха] ', '😨': ' [страх] ',
    '😰': ' [нервное_напряжение] ', '😥': ' [облегчение] ', '🤫': ' [тишина] ',
    '🤮': ' [рвота] ', '🥴': ' [опьянение] ', '😈': ' [ухмылка] ',
    '🥰': ' [любвеобильность] ', '😺': ' [кот] ', '🤡': ' [клоун] ',
    '🙄': ' [закатывание] ', '🔵': ' [синий] ', '🫠': ' [плавиться] ',
    '😦': ' [удивление] ', '🚉': ' [вокзал] ', '🚊': ' [трамвай] ',
    '🚌': ' [автобус] ', '🚲': ' [велосипед] ', '🚗': ' [автомобиль] ',
    '⏰': ' [будильник] ', '💰': ' [деньги] ', '💸': ' [деньги_улетают] ',
    '🎫': ' [билет] ', '🧳': ' [багаж] ', '👮': ' [полиция] ',
    '👷': ' [рабочий] ', '🛠️': ' [инструменты] ', '🚧': ' [ремонт] ',
    '⚠️': ' [предупреждение] ', '🚨': ' [сирена] ', '💺': ' [сиденье] ',
    '🔊': ' [громкость] ', '📢': ' [объявление] ', '📱': ' [телефон] ',
    '📶': ' [связь] ', '🙂': ' [улыбка] ', '🙃': ' [перевернутая_улыбка] ',
    '😌': ' [облегчение] ', '😇': ' [ангел] ', '😗': ' [поцелуй] ',
    '😙': ' [поцелуй_в_щеку] ', '😚': ' [поцелуй_с_закрытыми_глазами] ',
    '😋': ' [вкусно] ', '😛': ' [язык] ', '😝': ' [высунутый_язык] ',
    '😜': ' [подмигивание_язык] ', '🤪': ' [сумасшедший] ', '🤓': ' [ботаник] ',
    '😎': ' [крутой] ', '🤩': ' [обожание] ', '🥸': ' [переодевание] ',
    '🥳': ' [вечеринка] ', '🙂‍↕️': ' [голова_вверх_вниз] ', '🙂‍↔️': ' [голова_из_стороны_в_сторону] ',
    '😞': ' [разочарование] ', '😟': ' [озабоченность] ', '😕': ' [недоумение] ',
    '🙁': ' [легкая_грусть] ', '☹️': ' [хмурость] ', '😣': ' [страдание] ',
    '😖': ' [отчаяние] ', '😫': ' [усталость] ', '🥺': ' [умоляюще] ',
    '😤': ' [триумф] ', '🥵': ' [жарко] ', '😶‍🌫️': ' [туман] ',
    '🫣': ' [подглядывание] ', '🤭': ' [сдерживание_смеха] ', '🫢': ' [удивление_с_руками] ',
    '🫡': ' [салют] ', '🤥': ' [ложь] ', '😶': ' [молчание] ',
    '🫥': ' [исчезновение] ', '🫤': ' [неловкость] ', '😑': ' [равнодушие] ',
    '🫨': ' [тряска] ', '😬': ' [гримаса] ', '😯': ' [изумление] ',
    '😲': ' [удивление] ', '😮': ' [открытый_рот] ', '😧': ' [агония] ',
    '🤤': ' [слюни] ', '🥱': ' [зевота] ', '😴': ' [сон] ',
    '🫩': ' [шепот] ', '😵': ' [головокружение] ', '😮‍💨': ' [облегчение] ',
    '😪': ' [сонливость] ', '🤐': ' [рот_на_замке] ', '😷': ' [маска] ',
    '🤒': ' [термометр] ', '🤕': ' [перевязка] ', '🤠': ' [ковбой] ',
    '👿': ' [злой_бес] ', '👺': ' [гоблин] ', '👹': ' [огр] ',
    '💩': ' [какашка] ', '👻': ' [привидение] ', '💀': ' [череп] ',
    '☠️': ' [череп_и_кости] ', '👽': ' [инопланетянин] ', '😾': ' [сердитый_кот] ',
    '😼': ' [хитрый_кот] ', '😻': ' [влюбленный_кот] ', '😿': ' [плачущий_кот] ',
    '🙀': ' [испуганный_кот] ', '😹': ' [смеющийся_кот] ', '😸': ' [улыбающийся_кот] ',
    '😽': ' [целующий_кот] ', '🎃': ' [тыква] ', '🫶': ' [руки_сердце] ',
    '🧠': ' [мозг] ', '🤦‍♀️': ' [рука_к_лицу_женщина] ', '🤷‍♀️': ' [пожимание_плечами_женщина] ',
    '🙈': ' [не_видеть] ', '🌚': ' [луна] ', '💦': ' [пот] ',
    '💛': ' [сердце] ', '🧡': ' [сердце] ', '🩷': ' [сердце] ',
    '🩵': ' [сердце] ', '💙': ' [сердце] ', '💜': ' [сердце] ',
    '🖤': ' [сердце] ', '🩶': ' [сердце] ', '🤍': ' [белое_сердце] ',
    '🤎': ' [сердце] ', '💔': ' [разбитое_сердце] ', '❤️‍🔥': ' [пылающее_сердце] ',
    '🇷🇺': ' [флаг_России] '
    }
    
    for emoji, replacement in emoji_replacements.items():
        text = text.replace(emoji, replacement)
    
    # Эмотиконы
    emoticon_patterns = [
        (r':-\)|:\)|:\]|:D|:d', ' [улыбка] '),
        (r':-\(|:\(|:\[', ' [грусть] '),
        (r';-\)|;\)', ' [подмигивание] '),
        (r':-P|:P|:-p|:p', ' [язык] '),
        (r':-O|:O', ' [удивление] '),
        (r'>:\(', ' [злость] '),
        (r'<3', ' [сердце] '),
        (r':-\||:\|', ' [нейтрально] '),
        (r'xD|XD', ' [смех] '),
        (r'\.\.\.', ' [без_комментариев] '),
        (r'\)\)+|\)\)\)+', ' [сильно_ухмыляться] '), 
        (r'\(\(+|\(\(\(+', ' [сильно_грустить] '),  
    ]
    
    for pattern, replacement in emoticon_patterns:
        text = re.sub(pattern, replacement, text)
    
    return text

def normalize_metro_terms(text: str) -> str:
    """Нормализует термины, связанные с метро и транспортом"""
    # Словарь для замены (приведи к нижнему регистру для поиска)
    metro_dict = {
        # Организации
        'ржд': 'РЖД', 'жд': 'ЖД', 'цппк': 'ЦППК', 'мцд': 'МЦД', 'мцк': 'МЦК',
        'мосметро': 'Мосметро', 'мострансавто': 'Мострансавто',
        'аэроэкспресс': 'Аэроэкспресс',
        
        # Типы транспорта
        'электричка': 'электричка', 'экспресс': 'экспресс',
        'ласточка': 'Ласточка', 'иволга': 'Иволга', 'сапсан': 'Сапсан',
        'скорый': 'скорый', 'пассажирский': 'пассажирский',
        
        # Станции метро (примеры)
        'киевская': 'Киевская', 'белорусская': 'Белорусская',
        'павелецкая': 'Павелецкая', 'курская': 'Курская',
        'комсомольская': 'Комсомольская',
        
        # Термины
        'турникет': 'турникет', 'перрон': 'перрон', 'платформа': 'платформа',
        'вестибюль': 'вестибюль', 'тамбур': 'тамбур', 'вагон': 'вагон',
        'контролёр': 'контролёр', 'машинист': 'машинист',
        
        # Вокзалы
        'киевский': 'Киевский', 'белорусский': 'Белорусский',
        'павелецкий': 'Павелецкий', 'курский': 'Курский',
        'казанский': 'Казанский', 'ленинградский': 'Ленинградский',
        'ярославский': 'Ярославский',
    }
    
    # Простая замена слов (сохраняя регистр остального текста)
    words = text.split()
    result_words = []
    
    for word in words:
        lower_word = word.lower()
        # Проверяем полное совпадение слова
        if lower_word in metro_dict:
            # Заменяем на нормализованный вариант
            result_words.append(metro_dict[lower_word])
        else:
            # Ищем частичные совпадения для составных слов
            replaced = False
            for term, normalized in metro_dict.items():
                if term in lower_word and len(term) > 2:
                    # Для простоты заменяем целиком
                    result_words.append(normalized)
                    replaced = True
                    break
            if not replaced:
                result_words.append(word)  # оставляем как есть
    
    return ' '.join(result_words)

def preprocess_text(text: str) -> str:

    if not isinstance(text, str):
        return ""
    
    # Применяем все шаги по порядку
    text = clean_html(text)
    text = remove_urls_and_emails(text)
    text = process_emojis_and_emoticons(text)
    text = normalize_metro_terms(text)
    text = normalize_whitespace(text)
    
    return text

In [77]:
with open('data/vk_comments_with_metadata.json', 'r', encoding='utf-8') as f:
    comments = json.load(f)
    
len(comments)

14262

In [78]:
processed_comments = []
    
for i, comment in enumerate(comments):
    
    processed_comment = comment.copy()
    
    original_text = comment['text']
    cleaned_text = preprocess_text(original_text)
    
    processed_comment['text'] = cleaned_text
    
    processed_comments.append(processed_comment)

In [79]:
with open('data/vk_comments_preprocessed.json', 'w', encoding='utf-8') as f:
    json.dump(processed_comments, f, ensure_ascii=False, indent=2)

## Разметка

Мы предобработали текст, далее нужно добавить поле label, чтобы разметить где негатив нейтрал и позитив

```
    0: "negative",    # Негативный / жалоба / критика
    1: "neutral",     # Нейтральный / факт / вопрос
    2: "positive"     # Позитивный / похвала / благодарность
```

In [1]:
with open('data/vk_comments_preprocessed.json', 'r', encoding='utf-8') as f:
    comments = json.load(f)

NameError: name 'json' is not defined

In [81]:
transformed_comments = []

for i, comment in enumerate(comments):
    text = comment.get("text", "").strip()

    if not text or text == "" or text == " " or len(text) == 0:
        continue

    transformed = {
        "text": comment["text"],
        "label": "", 
        "metadata": {
            "group_id": comment["metadata"]["group_id"],
            "is_frequent_poster": comment["metadata"]["is_frequent_poster"]
        }
    }
    transformed_comments.append(transformed)

In [82]:
len(transformed_comments)

12714

In [83]:
with open("data/vk_comments_without_labels.json", 'w', encoding='utf-8') as f:
    json.dump(transformed_comments, f, ensure_ascii=False, indent=2)

# Загрузка и доработка размеченного датасета

In [7]:
emoji_replacements = {
    '👆': ' [палец_вверх] ',
    '😄': ' [радость] ',
    '👊': ' [кулак] ',
    '🖕': ' [средний_палец] ',
    '🍑': ' [персик] ',
    '🤦🏼‍♀️': ' [рука_к_лицу_женщина] ',
    '💪🏻': ' [мускулы] ',
    '🍄': ' [гриб] ',
    '😃': ' [радостная_улыбка] ',
    '🤝': ' [рукопожатие] ',
    '👏': ' [аплодисменты] ',
    '♥️': ' [сердце] ',
    '💥': ' [взрыв] ',
    '✅': ' [галочка] ',
    '❌': ' [крестик] ',
    '🚚': ' [грузовик] ',
    '✨': ' [блеск] ',
    '💋': ' [губы] ',
    '💯': ' [100_очков] ',
    '👇': ' [палец_вниз] ',
    '😆': ' [смех] ',
    '😠': ' [злость] ',
    '❤️': ' [сердце] ',
    '🌲': ' [дерево] ',
    '🐱': ' [кот] ',
    '🏆': ' [трофей] ',
    '🐘': ' [слон] ',
    '👌': ' [окей] ',
    '👌🏻': ' [окей] ',
    '💪': ' [мускулы] '
}

In [8]:
with open('data/vk_comments_with_labels.json', 'r', encoding='utf-8') as f:
        data = json.load(f)

In [9]:
for i, item in enumerate(data):
    original_text = item.get("text", "")
    fixed_text = original_text
    
    # Заменяем эмодзи
    for emoji, replacement in emoji_replacements.items():
        if emoji in fixed_text:
            fixed_text = fixed_text.replace(emoji, replacement)
            
    if fixed_text != original_text:
        item["text"] = fixed_text

In [10]:
with open('data/final_data.json', 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=2)

# Дообучение модели

Взял DeepPavlov/rubert-base-cased т.к. у меня gtx1650, круче модели будут больно видюхе делать

In [11]:
import json

In [12]:
with open('data/final_data.json', 'r', encoding='utf-8') as f:
    comments = json.load(f)

In [13]:
from sklearn.model_selection import train_test_split
import numpy as np

In [14]:
labels = [item['label'] for item in data]
texts = [item['text'] for item in data]

In [15]:
train_val_data, test_data, train_val_labels, test_labels = train_test_split(
    data, 
    labels,
    test_size=0.1,  # 10% для теста (400 примеров)
    random_state=42,
    stratify=labels  # стратификация по классам
)

In [16]:
train_labels = [item['label'] for item in train_val_data]
train_data, val_data = train_test_split(
    train_val_data,
    test_size=0.111,  # чтобы получить val = 10% от общего (400 примеров)
    random_state=42,
    stratify=train_labels  # ← снова стратификация
)

In [17]:
print(f"Train: {len(train_data)} ({len(train_data)/len(data)*100:.1f}%)")
print(f"Val: {len(val_data)} ({len(val_data)/len(data)*100:.1f}%)")
print(f"Test: {len(test_data)} ({len(test_data)/len(data)*100:.1f}%)")

Train: 3209 (80.0%)
Val: 401 (10.0%)
Test: 402 (10.0%)


In [18]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    EarlyStoppingCallback
)
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import numpy as np
from dataclasses import dataclass
from typing import Optional
import json

Вайбкодим подготовку данных т.к. там какая-то ошибка непонятная была

In [30]:
# Попробуйте разные способы импорта
try:
    # Способ 1: Обычный импорт
    from datasets import Dataset
    print("✅ Dataset импортирован через 'from datasets import Dataset'")
except ImportError:
    try:
        # Способ 2: Полный импорт
        import datasets
        Dataset = datasets.Dataset
        print("✅ Dataset импортирован через 'import datasets'")
    except ImportError:
        # Способ 3: Если не установлена библиотека datasets
        print("❌ Библиотека 'datasets' не установлена")
        print("   Установите: pip install datasets")
        # Создаем простую альтернативу
        class SimpleDataset:
            def __init__(self, data):
                self.data = data
                self.column_names = list(data.keys()) if isinstance(data, dict) else []
            
            def __len__(self):
                return len(list(self.data.values())[0]) if self.data else 0
            
            def __getitem__(self, idx):
                return {key: values[idx] for key, values in self.data.items()}
        
        Dataset = SimpleDataset
        print("✅ Используется SimpleDataset (заглушка)")

✅ Dataset импортирован через 'from datasets import Dataset'


In [31]:
# Создаем маппинг меток
all_labels = []
for data in [train_data, val_data, test_data]:
    all_labels.extend([item['label'] for item in data])

unique_labels = sorted(set(all_labels))
label2id = {label: i for i, label in enumerate(unique_labels)}
id2label = {i: label for label, i in label2id.items()}

print(f"Найдено классов: {len(unique_labels)}")
print(f"Маппинг меток: {label2id}")

# Функция для создания Dataset
def create_dataset(data, tokenizer, max_length=256):
    texts = [item['text'] for item in data]
    labels = [label2id[item['label']] for item in data]
    
    # Токенизация
    encodings = tokenizer(
        texts,
        truncation=True,
        padding='max_length',  # изменено на 'max_length'
        max_length=max_length,
        return_tensors=None  # важно для Dataset
    )
    
    print(f"  Токенизация завершена, пример: {texts[0][:50]}...")
    print(f"  Длина input_ids: {len(encodings['input_ids'][0])}")
    
    # Создание Dataset
    dataset = Dataset.from_dict({
        'input_ids': encodings['input_ids'],
        'attention_mask': encodings['attention_mask'],
        'labels': labels
    })
    
    return dataset

# Создание датасетов
print("\nСоздание train dataset...")
train_dataset = create_dataset(train_data, tokenizer)

print("\nСоздание val dataset...")
val_dataset = create_dataset(val_data, tokenizer)

print("\nСоздание test dataset...")
test_dataset = create_dataset(test_data, tokenizer)

print(f"\n✅ Датасеты созданы:")
print(f"Train: {len(train_dataset)} примеров")
print(f"Validation: {len(val_dataset)} примеров")
print(f"Test: {len(test_dataset)} примеров")

# Проверка структуры
print("\nПроверка структуры train_dataset:")
print(f"Колонки: {train_dataset.column_names}")
print(f"Тип первого элемента input_ids: {type(train_dataset[0]['input_ids'])}")
print(f"Длина первого input_ids: {len(train_dataset[0]['input_ids'])}")

Найдено классов: 3
Маппинг меток: {0: 0, 1: 1, 2: 2}

Создание train dataset...
  Токенизация завершена, пример: Чехов ❤...
  Длина input_ids: 256

Создание val dataset...
  Токенизация завершена, пример: Хорошо хоть Кипелова ещё можно исполнять [хитрость...
  Длина input_ids: 256

Создание test dataset...
  Токенизация завершена, пример: Чем-то «Сибирскую» в Новосибе напоминает...
  Длина input_ids: 256

✅ Датасеты созданы:
Train: 3209 примеров
Validation: 401 примеров
Test: 402 примеров

Проверка структуры train_dataset:
Колонки: ['input_ids', 'attention_mask', 'labels']
Тип первого элемента input_ids: <class 'list'>
Длина первого input_ids: 256


In [32]:
tokenizer = AutoTokenizer.from_pretrained("DeepPavlov/rubert-base-cased")

In [20]:
model = AutoModelForSequenceClassification.from_pretrained(
    "DeepPavlov/rubert-base-cased",
    num_labels=len(set([d['label'] for d in train_data]))
)

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:  38%|###8      | 273M/714M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


c:\Users\ASUS\Desktop\Programming\Kursovie\Textovie_dannie_da_da_dannie\nlp_env\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\ASUS\.cache\huggingface\hub\models--DeepPavlov--rubert-base-cased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [21]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

In [22]:
training_args = TrainingArguments(
    output_dir="./metro_comments_bert",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    learning_rate=2e-5
)

In [33]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

trainer.train()

c:\Users\ASUS\Desktop\Programming\Kursovie\Textovie_dannie_da_da_dannie\nlp_env\Lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss


KeyboardInterrupt: 